In [ ]:
import polars as pl
import pandas as pd
import plotly.express as px
from utilsforecast.plotting import plot_series

In [ ]:
data = pl.read_parquet(
    "data/london_smart_meters/preprocessed/london_smart_meters_merged_block_0-7.parquet"
)
timestamp = data.group_by("LCLid").agg(
    pl.datetime_range(
        start=pl.col("start_timestamp"),
        end=pl.col("start_timestamp").dt.offset_by(
            pl.format("{}m", pl.col("series_length").sub(1).mul(30))
        ),
        interval="30m",
    ).alias("timestamp"),
)
data = timestamp.join(data, on="LCLid", how="inner")
data.head(5)

In [ ]:
data = (
    data.filter(pl.col("file").eq("block_7"))
    .select(
        [
            "LCLid",
            "Acorn",
            "Acorn_grouped",
            "timestamp",
            "energy_consumption",
            "holidays",
            "visibility",
            "windBearing",
            "temperature",
            "dewPoint",
            "pressure",
            "apparentTemperature",
            "windSpeed",
            "precipType",
            "icon",
            "humidity",
            "summary",
        ]
    )
    .explode(
        [
            "timestamp",
            "energy_consumption",
            "holidays",
            "visibility",
            "windBearing",
            "temperature",
            "dewPoint",
            "pressure",
            "apparentTemperature",
            "windSpeed",
            "precipType",
            "icon",
            "humidity",
            "summary",
        ]
    )
)